# Utilities

This notebook contains basic utilities functions for Constitutive Models. They have been developed by Pedro Arduino (University of Washington) and edited/ported into python by Justin Bonus (Oct. 2019).

Use ``%run YOURPATH/'Bounding Surface'/Utilities.ipynb`` at the start of your notebook to import these functions.

In [4]:
import numpy as np

In [5]:
def dev(A):
    # Returns the deviatoric portion of a second order tensor A
    if A.shape == (3, 3):
        #tensor represented as 3x3 matrix
        traceA = sum(A[i][i] for i in range(n))
        s = A - traceA/3 * np.eye(3)
    elif A.shape == (6,1) or A.shape == (1,6) or A.shape == (6,):
        #tensor represented as 6x1, 1x6, 6x, and transpose vectors
        s = A
        traceA = trace(A)
        #numpy has some issues with 1D arrays, had to index value of traceA for cases differently
        if A.shape == (6,1):
            svol = np.array([[traceA/3],[traceA/3],[traceA/3],[0],[0],[0]])
            s = s - svol  
        else:
            svol = np.array([traceA/3,traceA/3,traceA/3,0,0,0])
            s = s - svol
    elif A.shape == (3,1) or A.shape == (1,3) or A.shape(3,):
        #tensor represented as 3x1, 1x3, 3x, AKA principal stress form
        A = A.reshape(3,1)
        s = A
        traceA = trace(A)
        svol = np.array([[traceA/3],[traceA/3],[traceA/3]]).reshape(3,1)
        s = s - svol
        
    else:
        print('Error: Unsupported representation of second order tensor in dev() function')
    return s

In [6]:
def trace(A):
    # Returns the trace of a second order tensor A
    n=3 
    if A.shape == (3, 3):
        #tensor represented as 3x3 matrix
        val = sum(A[i][i] for i in range(n))
    elif A.shape == (6,1) or A.shape == (6,) or A.shape == (1,6):
        #tensor represented as 6x1 matrix
        A = A.reshape(6,1)
        val = float(sum(A[i] for i in range(n)))
    elif A.shape == (3,1) or A.shape == (1,3) or A.shape(3,):
        A = A.reshape(3,1)
        val = float(sum(A[i] for i in range(n)))
    else:
        print('Error: Unsupported representation of second order tensor in trace() function')
    return val

In [7]:
def GetIvol():
    Vol = np.zeros((6,6)) #4th Volumetric
    for x in range(0, 3):
        for y in range(0, 3):
            Vol[x,y] = 1/3
            Vol[x,y] = 1/3
    return Vol

In [8]:
def GetI():
    One = np.eye(6) #4th Identity
    return One

In [9]:
def GetIdev(): 
    #Create 6x6 4th order deviatoric, 1/2 tail values
    factor = 2
    one3rd = 1/3
    oneHalf = 0.5
    #Dev = np.array([[np.eye(3) - np.multiply(one3rd, np.ones((3,3))), np.zeros((3,3))],
    #                [np.zeros((3,3)), np.multiply(oneHalf, np.eye(3))]])
    Dev = GetI() - GetIvol() #4th Deviatoric
    for x in range (3,6):
        for y in range (3,6):
            Dev[x,y] = Dev[x,y]/factor
    return Dev

In [10]:
def GetIdev1(): 
    #Create 6x6 4th order deviatoric, 'factor' tail values
    one3rd = 1/3
    factor = 1
    #Dev = np.array([[np.eye(3) - np.multiply(one3rd, np.ones((3,3))), np.zeros((3,3))],
    #                [np.zeros((3,3)), np.multiply(factor, np.eye(3))]])
    Dev = I() - Ivol() #4th Deviatoric
    for x in range (3,6):
        for y in range (3,6):
            Dev[x,y] = Dev[x,y]/factor    
    return Dev

In [1]:
def GetCe(mStatic):
    # Create the Elastic 4th Order Modulus
    # elastic stiffness tensor (matrix_hh)
    a = 2*mStatic.G 
    b = mStatic.K - a/3 # K + 4/3G
    b = mStatic.E  / (3 - 6*mStatic.v) - a / 3;
    #mC = np.array([[np.multiply(a, np.eye(3)) + np.multiply(b, np.ones((3,3))),  np.zeros((3,3))],
    #   [np.zeros((3,3)) , np.multiply(mStatic.G, np.eye(3))]])
    mC = 3*mStatic.K*GetIvol() + 2*mStatic.G*GetIdev() 
    
    return mC

In [5]:
def GetJ2(A): 
    if A.shape == (3,3):
        #This is just to remain consistent with the MATLAB file, will revamp
        m = 1/3 * (A[0,0]+A[1,0]+A[2,0])
        hold = np.zeros((6,1))
        hold[0] = A[0,0] - m 
        hold[1] = A[1,0] - m
        hold[2] = A[2,0] - m
        out = 0.5 * (hold[0]*hold[0] + hold[1]*hold[1] + hold[2]*hold[2] + 2*(A[0,1]*A[0,1] + A[1,1]*A[1,1] + A[2,1]*A[2,1]))
        out = out[0]
    elif A.shape == (6,) or A.shape == (6,1) or A.shape == (1,6):    
        #J2 of 6x1 and 1x6 inputs
        A = A.reshape(6,1) #force 6x1 dimension for calculation
        m = 1/3 * (A[0] + A[1] + A[2])
        hold = np.zeros((6,1))
        hold[0] = A[0] - m 
        hold[1] = A[1] - m
        hold[2] = A[2] - m
        out = 0.5 * (hold[0]*hold[0] + hold[1]*hold[1] + hold[2]*hold[2] + 2*(A[3]*A[3] + A[4]*A[4] + A[5]*A[5]))
        out = out[0]
        
    elif A.shape == (3,) or A.shape == (3,1) or A.shape == (1,3):
        A = A.reshape(3,1)
        m = 1/3 * (A[0] + A[1] + A[2])
        hold = np.zeros((3,1))
        hold[0] = A[0] - m 
        hold[1] = A[1] - m
        hold[2] = A[2] - m
        out = 0.5 * float((hold[0]*hold[0] + hold[1]*hold[1] + hold[2]*hold[2])) #0.5 neccesary?

    return out

In [13]:
def innerProduct( X, Y, type):
    #inner = X.*Y for type = stress, strain, or neutral storage
    inner = 0.0
    if X.shape == (3, 3):
        #tensor represented as 3x3
        inner = np.multiply(X,Y) # Outputs (3,3) array
    elif X.shape == (6,1) or X.shape == (6,) or X.shape == (1,6):
        X = X.reshape(6,1)
        Y = Y.reshape(6,1)
        #Pseudo-switch
        switcher = {
            1: 2.0, #Stress
            2: 0.5, #Strain
            3: 1.0  #stress:strain
        }
        modifier = switcher.get(type, "Invalid type in innerProduct()")
        for i in range(0, len(X)):
            inner = inner + X[i]*Y[i]
            inner = inner[0] # Output scalar
            if i > 2:
                inner = inner + (modifier - 1.0)*X[i]*Y[i]
                inner = inner[0] # Output scalar
    elif X.shape == (3,1) or X.shape == (3,) or X.shape == (1,3):
        X = X.reshape(3,1)
        Y = Y.reshape(3,1)
        for i in range(0, len(X)):
            inner = inner + X[i]*Y[i]
            inner = inner[0]
    else:
        print('Unsupported representation of second order tensor in innerProduct()')
    return inner

In [14]:
def normE(A):
    #val = normE(A) returns the norm of a second order tensor A (strain type)
    if A.shape == (3,3):
        #tensor represented as 3x3
        A2 = np.zeros((3,3))
        A2 = sum(A[i]*A[i] for i in range(0,len(A)))
        val = np.sqrt(sum(A2))
    elif A.shape == (6,1) or A.shape == (6,) or A.shape == (1,6):
        A = A.reshape(6,1)
        A2 = np.zeros((6,1))
        for i in range(0,len(A)):
            A2[i] = A[i]*A[i] 
        val = np.sqrt(sum(A2[i] for i in range(0,3)) + 0.5*sum(A2[i] for i in range(3,6)))
    elif A.shape == (3,1) or A.shape == (3,) or A.shape == (1,3):
        #tensor represented as 3x1 or 1x3
        A = A.reshape(3,1)
        A2 = np.zeros(3).reshape(3,1)
        for i in range(0,len(A)):
            A2[i] = A[i]*A[i]
        val = np.sqrt(sum(A2[i] for i in range(0,3)))
    else:
        print('Unsupported representation of second order tensor in normE()')
        val = 0
    return val

In [15]:
def normS(A):
    #val = normS(A) returns the norm of a second order tensor A (stress type)
    if A.shape == (3,3):
        #tensor represented as 3x3
        A2 = np.zeros((3,3))
        A2 = sum(A[i]*A[i] for i in range(0,len(A)))
        val = np.sqrt(sum(A2))
    elif A.shape == (6,1) or A.shape == (6,) or A.shape == (1,6):
        #tensor represented as 6x1 or 1x6
        A = A.reshape(6,1)
        A2 = np.zeros(6).reshape(6,1)
        for i in range(0,len(A)):
            A2[i] = A[i]*A[i] 
        val = np.sqrt(sum(A2[i] for i in range(0,3)) + 2*sum(A2[i] for i in range(3,6)))
    elif A.shape == (3,1) or A.shape == (3,) or A.shape == (1,3):
        #tensor represented as 3x1 or 1x3
        A = A.reshape(3,1)
        A2 = np.zeros(3).reshape(3,1)
        for i in range(0,len(A)):
            A2[i] = A[i]*A[i]
        val = np.sqrt(sum(A2[i] for i in range(0,3)))
    else:
        print('Unsupported representation of second order tensor in normS()')
        val = 0
    return val

In [16]:
def vectorNorm(X, type):
    #norm(X) in type = stress, strain or neutral storage format
    vNorm = np.sqrt(innerProduct(X,X,type))
    return vNorm

In [17]:
def evalH(param, kappa):
    #s = evalH(param, kappa) returns the value of H=h*kappa^m (AKA exponential hardening moduli)
    if kappa < 0:
        s = 1.0e-10
        return s
    s = param.hh*(kappa)**param.mm
    return s

In [1]:
def convert2StressLike(A):
    #stress = normS(A) .. returns a stress like second order temsor A
    #                  (in stress type storage)
    if A.shape == (3,3):
        #tensor represented as 3x3 matrix
        hold = A
    elif A.shape == (6,1) or A.shape == (6,) or A.shape == (1,6):
        #tensor represented as 6x1 or 1x6
        A = A.reshape(6,1)
        hold = np.zeros((6,1))
        hold[0] = A[0]
        hold[1] = A[1]
        hold[2] = A[2]
        hold[3] = 0.5*A[3]
        hold[4] = 0.5*A[4]
        hold[5] = 0.5*A[5]
    elif A.shape == (3,1) or A.shape == (3,) or A.shape == (1,3):
        A = A.reshape(3,1)
        hold = np.zeros((3,1))
        hold[0] = A[0]/2
        hold[1] = A[1]/2
        hold[2] = A[2]/2
    
    else:
        print('Unsupported representation of second order tensor in convert2StressLike()')
    
    
    return hold

In [19]:
def hydroProjector(stress, type):
    # REDUNDANT
    # Project forms of stress vectors onto a plane orthogonal to the hydrostatic axis, centered at the origin
    # U_{proj-plane} = U - ((U dot n)/(||n||^2))n
    # U = [s_11, s_22, s_33, s_12, s_23, s_13]; n = Vector normal to \pi-plane
    # U = [s_1, s_2, s_3]
    #===================================
    #Type 0 for for top 3 cells of 6x1, 1 for bottom 3 cells of 6x1, 2 for 3x1 principal form
    #===================================
    #Be careful that the correct unit vector is being used or an offset to the projected plane will occur 
    #Not a major issue when viewing figure in 'ortho' 
    import numpy as np
    #stress = stress.reshape(1,6)
    def zero(stress):
        #Projecting 3x1 normal principal vectors onto the \Pi-plane
        #Converting to 6x1 in order to use other functions that are called
        zero.vec = np.array([1,1,1,0,0,0])
        zero.norm_vec = normS(zero.vec)
        zero.unit_vec = zero.vec/zero.norm_vec
        zero.norm_unit_vec = normS(zero.unit_vec)
        zero.proj_stress = stress - ((innerProduct(stress,zero.unit_vec,1))/(zero.norm_unit_vec**2))*zero.unit_vec
        return zero.proj_stress
    def one(stress):
        one.vec = np.array([0,0,0,1,1,1])
        one.norm_vec = normS(one.vec)
        one.unit_vec = one.vec/one.norm_vec
        one.norm_unit_vec = normS(one.unit_vec)
        one.proj_stress = stress - ((innerProduct(stress,one.unit_vec,1))/(one.norm_unit_vec**2))*one.unit_vec
        return one.proj_stress
    def two(stress):
        two.vec = np.array([1,1,1])
        two.norm_vec = normS(two.vec)
        two.unit_vec = two.vec/two.norm_vec
        two.norm_unit_vec = normS(two.unit_vec)
        two.proj_stress = stress - ((innerProduct(stress,two.unit_vec,1))/(two.norm_unit_vec**2))*two.unit_vec
        return two.proj_stress        
    
    switch_type = {
        1: zero,
        2: one,
        3: two
    }
    #Pseudo-switch
    case = switch_type.get(type+1, lambda: "Invalid projection type")
    case(stress) #Runs the projection function.
    proj_stress = case.proj_stress
    return proj_stress

In [20]:
def vectorAngle(vec1, vec2, type):
    # First take the dev() and hydroProjector() of the vectors if measuring
    # angle from deviatoric view
    #
    #if vec1.shape == (6,1) or vec1.shape(6,) or vec1.shape == (1,6):
    #theta = cos-1((u dot v)/(||u|| dot ||v||))
    switch_type = {
        1: 2, #Stress
        2: 0.5, #Strain
        3: 1 #Stress Strain
    }
    factor = switch_type.get(type, 'Invalid type')
    angle = float(np.arccos(innerProduct(vec1, vec2,1)/(normS(vec1)*normS(vec2)))*(180/np.pi)) #Degrees     
    #else:
    #angle = 'Incorrect input vector shape'
    return angle

In [1]:
def princVal(vec, type):
    #Based on the code of Robert Siegwart, 2019
    #Edited by Justin Bonus, July 2019
    #type = 0 returns normal principals
    #type = 1 returns shear principals
    
    if vec.shape == (3,3):
        S = vec
    elif vec.shape == (6,1) or vec.shape == (6,) or vec.shape == (1,6):  
        vec = vec.reshape(6,1)
        S = np.array([[vec[0], vec[3], vec[5]],
                       [vec[3], vec[1], vec[4]],
                       [vec[5], vec[4], vec[2]] ])
        S = S.reshape(3,3)
    else:
        print('Unsupported vector shape in principal()')
    
    e_val, e_vec = np.linalg.eig(S) #Solve for eigenvalues and eigenvectors
    p3, p2, p1 = np.sort(e_val)   #Sort smallest to largest
    #p1, p2, p3 = e_val
    
    if type == 1:
        p1 = (p1+p3)/2
        p2 = (p1+p2)/2
        p3 = (p2+p3)/2

    return p1, p2, p3

In [2]:
def princVec(vec, type):
    #Based on the code of Robert Siegwart, 2019
    #Edited by Justin Bonus, July 2019
    #type = 0 returns normal principals
    #type = 1 returns shear principals
    
    if vec.shape == (3,3):
        S = vec
    elif vec.shape == (6,1) or vec.shape == (6,) or vec.shape == (1,6):  
        vec = vec.reshape(6,1)
        S = np.array([[vec[0], vec[3], vec[5]],
                       [vec[3], vec[1], vec[4]],
                       [vec[5], vec[4], vec[2]] ]).reshape(3,3)

    else:
        print('Unsupported vector shape in principal()')
    
    e_val, e_vec = np.linalg.eig(S) #Solve for eigenvalues and eigenvectors
    p3, p2, p1 = np.sort(e_val)   #Sort smallest to largest
    #p1, p2, p3 = e_val
    e_val_l = e_val.tolist() #Python list
    p1_index, p2_index, p3_index = e_val_l.index(p1), e_val_l.index(p2), e_val_l.index(p3)
    p1_vec, p2_vec, p3_vec = e_vec[:,p1_index], e_vec[:,p2_index], e_vec[:,p3_index]
    
    if type == 1:
        tau1 = (p1+p3)/2
        tau2 = (p1+p2)/2
        tau3 = (p2+p3)/2
        p1_vec = (p1_vec + p3_vec)/np.linalg.norm(p1_vec+p3_vec)
        p2_vec = (p1_vec + p2_vec)/np.linalg.norm(p1_vec+p2_vec)
        p3_vec = (p2_vec + p3_vec)/np.linalg.norm(p2_vec+p3_vec)

    return p1_vec, p2_vec, p3_vec

In [4]:
def principal(vec):
    p1,p2,p3 = princVal(vec,0)
    prin = np.array([p1,p2,p3]).reshape(3,1)
    return prin

In [23]:
def sixToPrincipal(stress):
    # Invaraiant alternative to eigenvalue/vector method for converting 6x1 stress to 3x1 principal stress
    # Justin Bonus, Aug 15 2019
    # Check if stress is the zero vector
    if np.all(stress == 0):
        princ = np.array([0,0,0]).reshape(3,1)
    else:
        stress = stress.reshape(6,1)
        I1 = stress[0] + stress[1] + stress[2]
        I2 = stress[0]*stress[1] + stress[1]*stress[2] + stress[2]*stress[0] - stress[3]**2 - stress[4]**2 - stress[5]**2
        I3 = stress[0]*stress[1]*stress[2] - stress[0]*(stress[4]**2) - stress[1]*(stress[5]**2) - stress[2]*(stress[3]**2) + 2*stress[3]*stress[4]*stress[5]
        phi = (1/3)*np.arccos((2*I1**3 - 9*I1*I2 + 27*I3)/(2*(I1**2 - 3*I2)**(3/2)))
        p1 = (I1/3) + (2/3)*(np.sqrt(I1**2 - 3*I2))*np.cos(phi)
        p2 = (I1/3) + (2/3)*(np.sqrt(I1**2 - 3*I2))*np.cos(phi - 2*np.pi/3)
        p3 = (I1/3) + (2/3)*(np.sqrt(I1**2 - 3*I2))*np.cos(phi - 4*np.pi/3)
        princ = np.array([p1,p2,p3]).reshape(3,1)
    return princ

In [24]:
def nineToSix(stress):
    # Intakes 3x3 stress, outputs 6x1
    six = np.array([0,0,0,0,0,0]).reshape(6,1)
    six[0] = stress[0,0]; six[1] = stress[1,1]; six[2] = stress[2,2]
    six[3] = stress[0,1]; six[4] = stress[1,2]; six[5] = stress[0,2]
    return six

In [25]:
def sixToNine(stress):
    # Intakes 6x1 stress, outputs 3x3
    nine = np.array([[0,0,0],[0,0,0],[0,0,0]])
    nine[0,0] = stress[0]; nine[1,1] = stress[1]; nine[2,2] = stress[2]
    nine[0,1] = stress[3]; nine[1,2] = stress[4]; nine[0,2] = stress[5]
    nine[1,0] = stress[3]; nine[2,1] = stress[4]; nine[2,0] = stress[5]
    return nine

In [2]:
#!pip install numpy-quaternion
import quaternion as quat
import numpy as np

def quatRotator(v, axis, theta):
    # Based on the code of henneray Jun 23 '17 and Nathaniel Aug 19 '18
    # https://stackoverflow.com/questions/6802577/rotation-of-3d-vector
    
    # Uses quaternion methods to rotate 3D numpy vectors quickly
    # Input: 
    # v = vector, (3x1) or (1x3) shape
    # axis = vector to rotate around, (3x1) or (1x3) shape
    # theta = angle in radians to rotate clockwise by
    
    #import numpy as np
    #pip install numpy-quaternion
    #pip install numba
    #import quaternion as quat 

    #vector = np.array([0.] + v) #Add real component cell
    rot_axis = np.array([0.] + axis) #Add real component cell
    axis_angle = (theta*0.5) * rot_axis/np.linalg.norm(rot_axis)

    vec = quat.quaternion(*v)
    qlog = quat.quaternion(*axis_angle)
    q = np.exp(qlog)

    v_prime = q * vec * np.conjugate(q)
    v_prime = v_prime.imag.reshape(3,1) #Remove real component and reshape
    return v_prime

def quatHydroRotator(v,theta):
    # Faster method of quaternion rotation
    # Slightly more accurate due to explicitly written unitHydro
    # Rotates only around the hydrostatic axis
    
    #import numpy as np
    #pip install numpy-quaternion
    #pip install numba
    #import quaternion as quat
    
    unitHydro = np.array([[0.],[np.sqrt(1/3)],[np.sqrt(1/3)],[np.sqrt(1/3)]])
    #vector = np.array([0.] + v)
    axis_angle = (theta*0.5) * unitHydro
    
    vec = quat.quaternion(*v)
    qlog = quat.quaternion(*axis_angle)
    q = np.exp(qlog)
    
    v_rot = q * vec * np.conjugate(q)
    v_rot = v_rot.imag.reshape(3,1) # Remove real component and reshape
    return v_rot

In [3]:
def indexRotator(vector, theta, origin = [0,0]): 
    #rotate x,y CCCW around xo,yo by theta (rad)
    # Defaults to origin at (0,0)
    xo = origin[0]; yo = origin[1]
    x = vector[0]; y = vector[1]
    xr=np.cos(theta)*(x-xo)-np.sin(theta)*(y-yo) + xo
    yr=np.sin(theta)*(x-xo)+np.cos(theta)*(y-yo) + yo
    result = np.array([xr,yr]).reshape(2,1)
    return result